## SPELL CHECK

In [3]:
!wget http://www.norvig.com/big.txt
!wget http://www.norvig.com/spell-testset2.txt
!wget http://www.norvig.com/spell-testset1.txt

--2022-12-13 03:43:18--  http://www.norvig.com/big.txt
Resolving www.norvig.com (www.norvig.com)... 158.106.138.13
Connecting to www.norvig.com (www.norvig.com)|158.106.138.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6488666 (6.2M) [text/plain]
Saving to: ‘big.txt.1’

big.txt.1           100%[===================>]   6.19M  16.3MB/s    in 0.4s    

2022-12-13 03:43:19 (16.3 MB/s) - ‘big.txt.1’ saved [6488666/6488666]

--2022-12-13 03:43:19--  http://www.norvig.com/spell-testset2.txt
Resolving www.norvig.com (www.norvig.com)... 158.106.138.13
Connecting to www.norvig.com (www.norvig.com)|158.106.138.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7518 (7.3K) [text/plain]
Saving to: ‘spell-testset2.txt’

spell-testset2.txt  100%[===================>]   7.34K  --.-KB/s    in 0s      

2022-12-13 03:43:19 (126 MB/s) - ‘spell-testset2.txt’ saved [7518/7518]

--2022-12-13 03:43:19--  http://www.norvig.com/spell-testset1.txt
Resolving

In [4]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [5]:
len(WORDS)

32198

In [6]:
WORDS.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [12]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.process_time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.process_time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [13]:

spelltest(Testset(open('spell-testset1.txt')))
spelltest(Testset(open('spell-testset2.txt')))

75% of 270 correct (6% unknown) at 36 words per second 
68% of 400 correct (11% unknown) at 30 words per second 


In [14]:
test = "wrogn spellin"
words(test)


['wrogn', 'spellin']

In [16]:
for w in words(test):
    print(candidates(w))

{'wrong'}
{'spelling'}


In [17]:
for w in words(test):
    print(correction(w))

wrong
spelling
